In [1]:
# %pip install boto3==1.34.93 ipython --quiet

%load_ext autoreload
%autoreload 2

In [2]:
# import boto3
import json, re
# %time
# session = boto3.session.Session()
# bedrock_client = session.client(
#     service_name='bedrock-runtime',
#     region_name='eu-west-3',
# )

# # Model id and Claude config
# model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
# claude_config = {
#     'max_tokens': 4096,
#     'temperature': 0, 
#     'anthropic_version': '',
#     'stop_sequences': ['Human:']
# }

In [3]:
# Fill with your json sample files here
cdc_json_file_path = "/workspace/formatted_json_cdc2.json" 
cv_json_file_path = "/workspace/formatted_json_cv2.json"
rec_json_file_path = "/workspace/formatted_json_rec2.json"

# Open the JSON file and load its contents
with open(cdc_json_file_path, "r") as file:
    cdc_json_data = json.load(file)
with open(cv_json_file_path, "r") as file:
    cv_json_data = json.load(file)
with open(rec_json_file_path, "r") as file:
    rec_json_data = json.load(file)

# Now you can work with the loaded JSON data
print(cdc_json_data)
print(cv_json_data)

# Add you json inputs here
REQUIREMENTS = cdc_json_data
RESUME = cv_json_data

{'objectif_de_la_mission': ["Mission permettant de valoriser les services SI et de garantir une forte valeur ajoutée à ses Clients sur les activités d'assistance à maîtrise d'ouvrage sur les applications du pôle Services FM.", 2], 'duree_de_la_mission': [216, 2], 'nombre_de_profils': [2, 3], 'types_de_profils': [{'type': ['101.S.PAR (en priorité)', 3]}, {'type': ['101.E.PAR', 3]}], 'taches_et_responsabilites': [{'tache_et_responsabilite': ['Gestion opérationnelle du projet (organisation, cadrage, planification, coordination, reporting, TQM, gestion documentaire, …)', 7]}, {'tache_et_responsabilite': ['Assistance au pilotage budgétaire des projets (suivi de la gestion des budgets : réalisé /budget, accostage, etc. …)', 7]}, {'tache_et_responsabilite': ['Reporting avancement auprès du responsable du pôle', 7]}, {'tache_et_responsabilite': ["A l'identification et la qualification des besoins et des enjeux", 7]}, {'tache_et_responsabilite': ["Au recueil, à la rédaction d'expressions de bes

In [4]:
CRITERIAS  = rec_json_data['sous_criteres_techniques_et_ponderation']

for critere_json in rec_json_data['sous_criteres_techniques_et_ponderation']:
    print(critere_json['critere'][0])

Qualité de la réponse (Compréhension du besoin, le cas échéant outils et méthodologie proposés)
Nombre et pertinence des expériences similaires
Compétences techniques
Complexité des missions gérées
Connaissance du domaine fonctionnel (Facility management)


In [5]:
SCALE = {
    0: "Ne répond pas au besoin",
    1: "Ne répond pas suffisamment au besoin",
    2: "Répond partiellement au besoin",
    3: "Répond correctement au besoin",
    4: "Très bonne réponse",
    5: "Réponse au-delà des attentes"
}


In [6]:
# prompt = f"""
# Vous êtes un assistant d'analyse des compétences techniques de CV dans le cadre de réponse à appel
# d'offre du groupe SNCF. Votre tâche est d'évaluer un CV par rapport à un ensemble de critères
# techniques pondérés fournis.

# Voici les informations dont vous disposez :

# <Requirements>
# {REQUIREMENTS}
# </Requirements>

# <Criteria>
# {CRITERIAS}
# </Criteria>

# <Resume>
# {RESUME}
# </Resume>

# <Scale>
# {SCALE}
# </Scale>
# """ + """
# Commencez par lire attentivement les exigences et les critères. Ensuite, analysez le CV par rapport
# à chaque critère. Pour chaque critère, fournissez d'abord une justification détaillée de la note que
# vous attribuez au candidat. Utilisez les informations du CV pour expliquer pourquoi vous pensez que
# le candidat mérite cette note pour ce critère spécifique.

# Après avoir fourni la justification, donnez une note numérique entre 1 et 5 en vous basant sur
# l'échelle de notation fournie.

# Une fois que vous avez analysé tous les critères, produisez un JSON avec les notes et les
# justifications pour chaque critère.

# Le format du JSON doit être le suivant :

# {
# 'critère 1': {
# 'justification': 'Justification détaillée pour la note',
# 'score': X
# },
# 'critère 2': {
# 'justification': 'Justification détaillée pour la note',
# 'score': Y
# },
# ...
# }

# Remplacez "critère 1", "critère 2", etc. par les noms réels des critères. Assurez-vous que les
# justifications sont claires et détaillées, et que les scores sont conformes à l'échelle de notation
# fournie.

# <scratchpad>
# Vous pouvez utiliser cette zone pour prendre des notes et organiser vos pensées avant de produire le
# JSON final.
# </scratchpad>

# <json_output>
# Écrivez votre sortie JSON ici.
# </json_output>
# """

In [7]:
# def prepare_bedrock_request(messages, claude_config):
#     return {
#         'messages': messages,
#         **claude_config,
#     }
# def extract_bedrock_response(response):
#     body = json.loads(response['body'].read().decode('utf-8'))
#     return body['content'][0]['text']

# def pretty_print(message):
#     print('\n\n'.join('\n'.join(line.strip() for line in re.findall(r'.{1,100}(?:\s+|$)', paragraph.strip('\n'))) for paragraph in re.split(r'\n\n+', message)))

In [8]:
# %%time
# messages = [
#     {"role": "user", "content": [
#         {"type": "text", "text": prompt},
#         ]
#     }
# ]

# body = prepare_bedrock_request(messages, claude_config)
# response = bedrock_client.invoke_model(modelId=model_id, body=json.dumps(body))
# formated_response = extract_bedrock_response(response)

In [9]:
# pretty_print(formated_response)

In [10]:
from dag_nao.modules.technical_notation.compare_experiences.llm_comparator import LLMComparator

llm_comparator = LLMComparator()
formated_response, formated_json_response = llm_comparator.call_comparator(requirements=REQUIREMENTS, criterias=CRITERIAS, resume=RESUME, scale=SCALE)

In [14]:
formated_response

'Voici mon analyse du CV par rapport aux critères fournis, avec les justifications détaillées et les scores correspondants :\n\n<json_output>\n{\n"Qualité de la réponse (Compréhension du besoin, le cas échéant outils et méthodologie proposés)": {\n"justification": "Le CV ne fournit pas d\'informations spécifiques sur la compréhension du besoin, les outils ou les méthodologies proposées pour cette mission. Cependant, les expériences professionnelles décrites montrent une bonne capacité à comprendre les besoins des clients et à proposer des solutions adaptées.",\n"score": 3\n},\n"Nombre et pertinence des expériences similaires": {\n"justification": "Le candidat possède une vaste expérience de plus de 30 ans dans le développement et la gestion de projets liés aux ERP, notamment dans le domaine immobilier et de la gestion locative sociale. Ses missions chez SCEPIA, éditeur d\'ERP pour les bailleurs sociaux, et chez GOFOR, éditeur d\'ERP pour l\'habillement, sont particulièrement pertinente

In [11]:
formated_json_response

{'Qualité de la réponse (Compréhension du besoin, le cas échéant outils et méthodologie proposés)': {'justification': "Le CV ne fournit pas d'informations spécifiques sur la compréhension du besoin, les outils ou les méthodologies proposées pour cette mission. Cependant, les expériences professionnelles décrites montrent une bonne capacité à comprendre les besoins des clients et à proposer des solutions adaptées.",
  'score': 3},
 'Nombre et pertinence des expériences similaires': {'justification': "Le candidat possède une vaste expérience de plus de 30 ans dans le développement et la gestion de projets liés aux ERP, notamment dans le domaine immobilier et de la gestion locative sociale. Ses missions chez SCEPIA, éditeur d'ERP pour les bailleurs sociaux, et chez GOFOR, éditeur d'ERP pour l'habillement, sont particulièrement pertinentes. De plus, il a occupé des rôles de chef de projet, directeur technique et product owner, ce qui correspond aux responsabilités attendues.",
  'score': 5

In [12]:
CRITERIAS

[{'critere': ['Qualité de la réponse (Compréhension du besoin, le cas échéant outils et méthodologie proposés)',
   1],
  'ponderation': [15, 1]},
 {'critere': ['Nombre et pertinence des expériences similaires', 1],
  'ponderation': [20, 1]},
 {'critere': ['Compétences techniques', 1], 'ponderation': [30, 1]},
 {'critere': ['Complexité des missions gérées', 1], 'ponderation': [20, 1]},
 {'critere': ['Connaissance du domaine fonctionnel (Facility management)', 1],
  'ponderation': [15, 1]}]

In [13]:
# Create a dictionary to store the aggregated scores
aggregated_scores = 0
criteria_scores = formated_json_response
criteria_weights = CRITERIAS
# Iterate through each criterion and its weight
for criterion in criteria_weights:
    critere_name = criterion['critere'][0]
    weight = criterion['ponderation'][0] / 100

    # Iterate through the scores and justifications
    for key, value in criteria_scores.items():
        if key == critere_name:
            # Aggregate the weighted score
            aggregated_scores += value['score'] * (weight)

print(f"Note technique : {aggregated_scores}")

Note technique : 4.1
